In [3]:
import networkx as nx
import os


p = '../'
folderPath = p + 'dataset/graph files'
for i, file in enumerate(os.listdir(folderPath)):

            
            """
            If .pkl does not work there is always .net using G = nx.read_pajek(file_path) but you need
            to do the mapping of nodes from strings to int and furthermore in order to access the edge weight
            the following code need to changed from G.get_edge_data(paths[i], paths[i+1])['weight'] to 
            G.get_edge_data(paths[i], paths[i+1])[0]['weight']
            """
            
            if file.endswith(".net"):
                instanceName = file[0:len(file)-4]
                index = int(file.split('.')[0])
                # print(index)
                if index != 0:
                # if True:
                    file_path = f"{folderPath}/{file}"
                    G = nx.read_pajek(file_path)
                    print(G, type(G))
                    print(G.nodes(data=True))
                    break
                    


MultiGraph with 12 nodes and 26 edges <class 'networkx.classes.multigraph.MultiGraph'>
[('1', {'id': '1', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('2', {'id': '2', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('3', {'id': '3', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('4', {'id': '4', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('5', {'id': '5', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('6', {'id': '6', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('7', {'id': '7', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('8', {'id': '8', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('9', {'id': '9', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('10', {'id': '10', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('11', {'id': '11', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'}), ('12', {'id': '12', 'x': 0.0, 'y': 0.0, 'shape': 'ellipse'})]


In [4]:
import networkx as nx
from math import exp

class RobustMagnetPlanner:
    """
    'Magnetic-field' heuristic for MD-RPP-RRV with two upgrades

    1. depot-incident required edges are dynamically down-weighted
       until most other work is finished;

    2. each required edge may have a multiplicity m >= 1
       (traverse-twice, three-times, ...).

    Parameters
    ----------
    G : networkx.Graph
        Must have 'weight' on every edge.
    depots : iterable
    capacity : float
    alpha : float, optional
        Exponential decay coefficient.
    eta : float in [0,1), optional
        Static dampening for *all* depot-edges (0 ⇒ symmetric magnets).
    eps : small float, optional
        Base pull for depot-edges when progress = 0.
    k_ramp : int ≥ 1, optional
        Controls how fast depot-edge pull increases with progress.
    delta_stick : float, optional
        Bonus when the robot is already on an edge that still has
        multiplicity > 0 (helps satisfy "traverse twice" locally).
    """
    def __init__(
        self, G, depots, capacity,
        alpha=0.7, eta=0.4,
        eps=0.05, k_ramp=2,
        delta_stick=0.05
    ):
        self.G       = G
        self.D       = set(depots)
        self.C       = capacity
        self.alpha   = alpha
        self.eta     = eta
        self.eps     = eps
        self.k       = k_ramp
        self.delta   = delta_stick

        # All-pairs shortest paths / lengths
        self.dist  = dict(nx.all_pairs_dijkstra_path_length(G, weight="weight"))
        self.path  = dict(nx.all_pairs_dijkstra_path(G, weight="weight"))

    # ---------- helpers --------------------------------------------------

    def _edge_w(self, u, v):
        return self.G[u][v]["weight"]

    def _p_len(self, a, b):
        return self.dist[a][b]

    def _p_nodes(self, a, b):
        return self.path[a][b]

    def _nearest_depot_dist(self, node):
        return min(self._p_len(node, d) for d in self.D)

    # ---------- public interface ----------------------------------------

    def solve(self, required_edge_list):
        """
        required_edge_list : list[tuple[u,v]]  (duplicates ⇒ multiplicity>1)
        returns list of trips; each trip == (node_sequence, cost)
        """
        # build multiplicity dict  {edge_key: remaining_copies}
        mult = {}
        for u, v in required_edge_list:
            key = tuple(sorted((u, v)))
            mult[key] = mult.get(key, 0) + 1

        total_copies = sum(mult.values())
        trips = []

        while mult:
            start = min(self.D)        # any depot works – pick the smallest id
            trip_nodes, trip_cost, done = self._one_trip(start, mult, total_copies)
            trips.append((trip_nodes, trip_cost))
            # remove edges whose multiplicity hit 0 inside _one_trip
            for e in done:
                if mult[e] == 0:
                    del mult[e]

        return trips

    # ---------- single-trip builder -------------------------------------

    def _one_trip(self, start, mult, total_copies):
        here   = start
        route  = [start]
        cost   = 0.0
        used   = set()          # edges whose mult hit 0 in this trip

        while mult:
            remaining = sum(mult.values())
            progress  = 1.0 - remaining / total_copies  # p in [0,1]

            best = None  # (score, enter, exit, edge_key)

            for key, m in mult.items():
                a, b = key
                for enter, exit_ in ((a, b), (b, a)):
                    d_in   = self._p_len(here, enter)
                    w_edge = self._edge_w(a, b)
                    d_out  = self._nearest_depot_dist(exit_)

                    if cost + d_in + w_edge + d_out > self.C:
                        continue  # infeasible with current battery

                    base_attr = exp(-self.alpha * d_in)

                    dep_incident = (enter in self.D) or (exit_ in self.D)
                    if dep_incident:
                        base_attr *= (self.eps + progress ** self.k) * (1 - self.eta)

                    if here in (a, b):   # already on the edge?
                        base_attr += self.delta

                    if best is None or base_attr > best[0]:
                        best = (base_attr, enter, exit_, key)

            # no feasible required edge copy left → go home
            if best is None:
                home = min(self.D, key=lambda d: self._p_len(here, d))
                route += self._p_nodes(here, home)[1:]
                cost  += self._p_len(here, home)
                return route, cost, used

            # commit to best edge
            _, enter, exit_, key = best
            a, b   = key
            route += self._p_nodes(here, enter)[1:]
            cost  += self._p_len(here, enter)
            route.append(exit_)                        # traverse required edge
            cost  += self._edge_w(a, b)

            mult[key] -= 1
            if mult[key] == 0:
                used.add(key)
            here = exit_

        # all copies covered inside this trip → return to depot
        home = min(self.D, key=lambda d: self._p_len(here, d))
        route += self._p_nodes(here, home)[1:]
        cost  += self._p_len(here, home)
        return route, cost, used
